# Description

This notebook uses the median- and quantile-derived scale factors ($\beta$ values, outputs from [MedianScaling_calculate_beta.ipynb](./MedianScaling_calculate_beta.ipynb) and [QuantileScaling_calculate_beta.ipynb](./QuantileScaling_calculate_beta.ipynb), respectively) to scale the climatological distribution (CLIM) of maximum temperatures (defined as the collection of observations over 1985-2015) at each location globally and for each month of the year.

The resulting forced and counterfactual distributions of maximum temperatures are used in the the observation-based attribution analysis. Example attributuon analyses with these output distributions at Phoenix, AZ, USA, are provided in XXXXXXXX.

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import utilities as util
import analysisfx as analysis

# ignore depreciation warnings in this code
import warnings
warnings.filterwarnings("ignore")

# Load & Organize Data

### Berkeley daily maximum temperatures

In [2]:
# define the berkeley daily maximum temperature data pathway
berkdpath='./data/berkeley/Berkeley_TMAX_land_188001_201712_daily_N96_365days_degC.nc'
# load in the daily maximum temperature rasters 
bddat=xr.open_dataset(berkdpath)

#### Organize time

In [3]:
# create the datetime grid for Berkeley monthly and daily data
timegridbd=util.get_dt64(bddat)

# add these back into the data structures
bddat['time']=timegridbd

#### Replace 90S invalid data with missing

In [4]:
# set 90S equal to missing
bddat=util.set_N96_SouthPole_missing(bddat,'TMAX')

### GMST (global monthly timeseries, 3-year mean filtered via hadcrut_GMST.ipynb)

In [5]:
# load the data
gmstpath='./data/analysis/GMST-3yrmean.nc'
gmst = xr.open_dataset(gmstpath)

## Load Median Scale Factors

In [6]:
# define the paths to the load the scale factors from
q50_beta_loadpath='./data/analysis/q50_beta.nc'
qxx_beta_loadpath='./data/analysis/qxx_mon{month:02d}_beta.nc'

# load the scale factors
# median-derived
q50_beta=xr.open_dataset(q50_beta_loadpath)

# (define the months array from loaded data)
monthsi=np.asarray(q50_beta.month.values)


# Find scenario Deltas from HadCRUT5 data

The scenarios for average temperature are:
* Counterfactual (1885-1915)
* Climate Base Period (1985-2015)
* IPCC Attributable Warming (2010-2019)

These periods and their relative global mean temperature anomalies will be used to, in part, drive the observation-based scaling method.

In [7]:
# calculate differences between periods
cf_shift=(gmst['Counterfactual_Delta']-gmst['CLIM_Delta']).values.flatten()[0]
forced_shift=(gmst['Attributable_Delta']-gmst['CLIM_Delta']).values.flatten()[0]
# and print results to screen
print('Counterfactual GMST (1885-1915) anomaly from CLIM (1985-2015): '+str(np.round(cf_shift,2))+'C')
print('IPCC Attributable Warming GMST (2010-2019) anomaly from CLIM (1985-2015): '+str(np.round(forced_shift,2))+'C')
diff_ATT_minus_CF=forced_shift-cf_shift
print('IPCC Attributable Warming minus Natural: +'+str(np.round(diff_ATT_minus_CF,2))+'C')

Counterfactual GMST (1885-1915) anomaly from CLIM (1985-2015): -0.85C
IPCC Attributable Warming GMST (2010-2019) anomaly from CLIM (1985-2015): 0.29C
IPCC Attributable Warming minus Natural: +1.13C


### Define the Climatological Distribution (CLIM) from Observed Berkeley Data

Let's load/define the Climatological distribution of maximum temperatures from 1985-2015.

In [8]:
# find the baseline distribution (1985-2015), subselect that daily distribution
CLIMperiod=[1985,2015]
CLIM_slice=util.dt64_yrslice(CLIMperiod[0],CLIMperiod[1])
CLIMdist=bddat.sel(time=CLIM_slice)

# get the indices associated with each month
CLIMmonidx=CLIMdist.groupby('time.month').groups

# view the dataset
CLIMdist

<xarray.Dataset>
Dimensions:  (lat: 145, lon: 192, time: 11315)
Coordinates:
  * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * time     (time) datetime64[ns] 1985-01-01 1985-01-02 ... 2015-12-31
Data variables:
    TMAX     (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan
Attributes:
    standard_name:  surface_temperature_anomaly
    long_name:      Air Surface Temperature
    units:          degree C

# Scale CLIM with Median scale factors ($\beta_{q_i=0.5}$)

In [9]:
# define the location to save the distributions to
q50savepath='./data/analysis/q50_mon{month:02d}_scaled_distributions.nc'

# loop over the months
for mi in monthsi:
    
    # create monthly distribution from baseline
    base=CLIMdist['TMAX'][CLIMmonidx[mi]].rename('CLIM')
    
    # find the scaling temperature anomalies
    scale_Tanom_forced=q50_beta.sel(month=mi)*forced_shift
    scale_Tanom_cf=q50_beta.sel(month=mi)*cf_shift
    
    # scale the distributions
    forced=(base+scale_Tanom_forced).rename({'beta':'forced'})
    cf=(base+scale_Tanom_cf).rename({'beta':'counterfactual'})
    del scale_Tanom_forced, scale_Tanom_cf
    
    # assign to a single dataset
    q50_scaled_dists_mi=xr.Dataset(
        {
            "base": (
                ("time","lat", "lon"),
                base.data,
            ),
            "forced": (
                ("time","lat", "lon"),
                forced['forced'].data,
            ),
            "counterfactual": (
                ("time","lat", "lon"),
                cf['counterfactual'].data,
            ),
        },
        coords={"lat": base.lat, "lon": base.lon, "time": base.time, "month": mi},
        attrs={"month": mi, "quantile": q50_beta['quantile'].values,
               "forced_period": '2010-2019', "counterfactual_period": '1885-1915', 
               "forced_Tshift": forced_shift, "counterfactual_Tshift": cf_shift, 
               "unit": 'degrees C'
              }
    )
    # add in the month
    q50_scaled_dists_mi=q50_scaled_dists_mi.expand_dims('month')
    
    # clean up
    del base, forced, cf

    # save out to a netcdf file, now that we have the Q50 data array
    q50_scaled_dists_mi.to_netcdf(q50savepath.format(month=mi))
    del q50_scaled_dists_mi
    
    # where are we in the loop?
    print('mon index='+str(mi))
    

mon index=1
mon index=2
mon index=3
mon index=4
mon index=5
mon index=6
mon index=7
mon index=8
mon index=9
mon index=10
mon index=11
mon index=12


# Scale CLIM with Quantile scale factors ($\beta_{q_i=\{0.1,0.4,...,0.96,0.99\}}$)

In [11]:
# define a path to load the quantile data from
qxx_Tquant_loadpath='./data/berkeley/qxx_mon{month:02d}_1880-2017.nc'

# define the location to save the distributions to
qxxsavepath='./data/analysis/qxx_mon{month:02d}_scaled_distributions.nc'

# quantile-derived (load by month and merge)
for mi in monthsi:
    
    # load the quantiles scaling factor data
    qxx_beta_in=xr.open_dataset(qxx_beta_loadpath.format(month=mi))
    # ensure month is a correct dimension
    if len(qxx_beta_in.dims)==3:
        qxx_beta_in=qxx_beta_in.assign_coords(month=mi).expand_dims('month')
    # initialize the quantiles we are calculating over
    if mi==1:
        qi=np.asarray(qxx_beta_in['quantile'].values)
        
    # load the raw quantile temperature data
    qxx_Tquant=xr.open_dataset(qxx_Tquant_loadpath.format(month=mi))
    
    # initialize the climatology
    # create monthly distribution from baseline
    base=qxx_Tquant.TMAX.sel(time=CLIM_slice).rename({'CLIM'})
        
    # find the scaling temperature anomalies
    scale_Tanom_forced=qxx_beta_in*forced_shift
    scale_Tanom_cf=qxx_beta_in*cf_shift
    
    # scale the distributions
    forced=(base+scale_Tanom_forced).rename({'beta':'forced'}).isel(month=0)
    cf=(base+scale_Tanom_cf).rename({'beta':'counterfactual'}).isel(month=0)
    del scale_Tanom_forced, scale_Tanom_cf
    
    # assign to a single dataset
    qxx_scaled_dists_mi=xr.Dataset(
        {
            "base": (
                ("time","quantile","lat", "lon"),
                base.data,
            ),
            "forced": (
                ("time","quantile","lat", "lon"),
                forced['forced'].data,
            ),
            "counterfactual": (
                ("time","quantile","lat", "lon"),
                cf['counterfactual'].data,
            ),
        },
        coords={"lat": base.lat, "lon": base.lon, "time": base.time, "month": mi},
        attrs={"month": mi, "quantile": qi,
               "forced_period": '2010-2019', "counterfactual_period": '1885-1915', 
               "forced_Tshift": forced_shift, "counterfactual_Tshift": cf_shift, 
               "unit": 'degrees C'
              }
    )
    # add in the month
    qxx_scaled_dists_mi=qxx_scaled_dists_mi.expand_dims('month')
    
    # clean up
    del base, forced, cf

    # save out to a netcdf file, now that we have the Q50 data array
    qxx_scaled_dists_mi.to_netcdf(qxxsavepath.format(month=mi))
    del qxx_scaled_dists_mi
    
    # where are we in the loop?
    print('mon index='+str(mi))
    
    # clean up
    del qxx_beta_in, qxx_Tquant

mon index=1
mon index=2
mon index=3
mon index=4
mon index=5
mon index=6
mon index=7
mon index=8
mon index=9
mon index=10
mon index=11
mon index=12
